In [1]:
%pwd


'/content'

In [2]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Note: using Google CoLab


In [3]:
# install required packages
!pip install openai
!pip install gradio

In [6]:
# import the packages
import openai
import gradio as gr
import json
from typing import List, Dict, Tuple

client = openai.OpenAI()

# Check if you have set your ChatGPT API successfully
# You should see "Set ChatGPT API sucessfully!!" if nothing goes wrong.
try:
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [{'role':'user','content': "test"}],
            max_tokens=1,
    )
    print("Set ChatGPT API sucessfully!!")
except:
    print("There seems to be something wrong with your ChatGPT API. Please follow our demonstration in the slide to get a correct one.")

Set ChatGPT API sucessfully!!


In [8]:
import gradio as gr
from openai import OpenAI
from typing import List, Tuple

client = OpenAI()

# ---- Conversation Memory ----
messages = []
chat_history = []   # <-- store chat UI history separately

# ---- Prompt for summarization ----
prompt_for_summarization = "Please summarize the following article carefully:"

# ---- Reset function ----
def reset() -> List:
    global messages, chat_history
    messages = []
    chat_history = []
    return []

# ---- Summarization interaction ----
def interact_summarization(prompt: str, article: str, temp=1.0) -> List[Tuple[str, str]]:
    global messages, chat_history
    input_text = f"{prompt}\n{article}"

    # Add to conversation memory
    messages.append({"role": "user", "content": input_text})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp,
        max_tokens=200,
    )

    reply = response.choices[0].message.content

    # Save assistant reply in memory
    messages.append({"role": "assistant", "content": reply})

    # Add to chatbot UI history
    chat_history.append((input_text, reply))

    return chat_history

# ---- General chatbot interaction ----
def interact_chatbot(user_input: str, temp=1.0) -> List[Tuple[str, str]]:
    global messages, chat_history

    # Add user message
    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp,
        max_tokens=200,
    )

    reply = response.choices[0].message.content

    # Save assistant message
    messages.append({"role": "assistant", "content": reply})

    # Add to chatbot UI history
    chat_history.append((user_input, reply))

    return chat_history

# ---- Gradio UI ----
with gr.Blocks() as demo:
    gr.Markdown("# Summarization + Chatbot")
    chatbot = gr.Chatbot()
    prompt_textbox = gr.Textbox(label="Prompt", value=prompt_for_summarization, visible=False)
    article_textbox = gr.Textbox(label="Article", interactive=True, value="Paste any article here...")
    user_input_textbox = gr.Textbox(label="Chatbot Input", interactive=True, value="Hello!")

    with gr.Column():
        gr.Markdown("### Temperature (higher = more creative)")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="Temperature")

    with gr.Row():
        summarize_button = gr.Button(value="Summarize Article")
        chat_button = gr.Button(value="Chat")
        reset_button = gr.Button(value="Reset Conversation")

    # Event bindings
    summarize_button.click(interact_summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])
    chat_button.click(interact_chatbot, inputs=[user_input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])

demo.launch(debug=True)


/tmp/ipython-input-1492695083.py:73: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c2e5881d9df0706d8a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://c2e5881d9df0706d8a.gradio.live
